<a href="https://colab.research.google.com/github/vincentmin/llama-2-qlora/blob/main/Evaluation_harness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/EleutherAI/lm-evaluation-harness
!pip install -qe lm-evaluation-harness

Cloning into 'lm-evaluation-harness'...
remote: Enumerating objects: 14128, done.
remote: Counting objects: 100% (2540/2540), done.
remote: Compressing objects: 100% (484/484), done.
remote: Total 14128 (delta 2264), reused 2153 (delta 2052), pack-reused 11588
Receiving objects: 100% (14128/14128), 19.21 MiB | 7.07 MiB/s, done.
Resolving deltas: 100% (9379/9379), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 32.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing meta

In [ ]:
pip install -q accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

Here we test the original llama-2-7b-chat-hf model. To keep the duration of the evaluation reasonable, we restrict to a subset of tasks and use zero-shot.

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
model_args = f"pretrained={model_name},load_in_4bit=True,bnb_4bit_compute_dtype=float16"
num_few_shot=0
tasks = 'hendrycksTest-abstract_algebra,hendrycksTest-college_computer_science,hendrycksTest-college_mathematics,hendrycksTest-college_physics,hendrycksTest-conceptual_physics,hendrycksTest-elementary_mathematics,hendrycksTest-formal_logic,hendrycksTest-global_facts,hendrycksTest-logical_fallacies,hendrycksTest-machine_learning'
!python lm-evaluation-harness/main.py \
  --model=hf-causal-experimental \
  --model_args=$model_args \
  --tasks=$tasks \
  --num_fewshot=$num_few_shot \
  --batch_size=64 \
  --output_path="hendrycks/llama2.json" \
  --device=cuda:0

2023-07-23 15:38:36.751808: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Selected Tasks: ['hendrycksTest-abstract_algebra', 'hendrycksTest-college_computer_science', 'hendrycksTest-college_mathematics', 'hendrycksTest-college_physics', 'hendrycksTest-conceptual_physics', 'hendrycksTest-elementary_mathematics', 'hendrycksTest-formal_logic', 'hendrycksTest-global_facts', 'hendrycksTest-logical_fallacies', 'hendrycksTest-machine_learning']
Loading checkpoint shards: 100% 2/2 [01:06<00:00, 33.11s/it]
Found cached dataset mmlu (/root/.cache/huggingface/datasets/cais___mmlu/abstract_algebra/1.0.0/5c9df7080598ff5f0cc84b0e0519cfdc5516c84877d7893e547389485cfa78d8)
100% 4/4 [00:00<00:00,  6.16it/s]
Found cached dataset mmlu (/root/.cache/huggingface/datasets/cais___mmlu/college_computer_science/1.0.0/5c9df7080598ff5f0cc84b0e0519cfdc5516c84877d7893e547389485cfa78d8)
100% 4/4 [00:00<00:00,  6.50it/s]
Found cached dataset mmlu (/root/.cache/hugging

Here we load our adapter and repeat the evaluation.

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
adapter = "vincentmin/llama-7b-orca"
model_args = f"pretrained={model_name},peft={adapter},load_in_4bit=True,bnb_4bit_compute_dtype=float16"
tasks = 'hendrycksTest-abstract_algebra,hendrycksTest-college_computer_science,hendrycksTest-college_mathematics,hendrycksTest-college_physics,hendrycksTest-conceptual_physics,hendrycksTest-elementary_mathematics,hendrycksTest-formal_logic,hendrycksTest-global_facts,hendrycksTest-logical_fallacies,hendrycksTest-machine_learning'
!python lm-evaluation-harness/main.py \
  --model=hf-causal-experimental \
  --model_args=$model_args \
  --tasks=$tasks \
  --num_fewshot=0 \
  --batch_size=8 \
  --output_path="hendrycks/llama2.json" \
  --device=cuda:0

2023-07-23 17:55:24.455541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Selected Tasks: ['hendrycksTest-abstract_algebra', 'hendrycksTest-college_computer_science', 'hendrycksTest-college_mathematics', 'hendrycksTest-college_physics', 'hendrycksTest-conceptual_physics', 'hendrycksTest-elementary_mathematics', 'hendrycksTest-formal_logic', 'hendrycksTest-global_facts', 'hendrycksTest-logical_fallacies', 'hendrycksTest-machine_learning']
Loading checkpoint shards: 100% 2/2 [01:13<00:00, 36.67s/it]
Dataset mmlu downloaded and prepared to /root/.cache/huggingface/datasets/cais___mmlu/abstract_algebra/1.0.0/5c9df7080598ff5f0cc84b0e0519cfdc5516c84877d7893e547389485cfa78d8. Subsequent calls will reuse this data.
100% 4/4 [00:00<00:00, 222.08it/s]
Dataset mmlu downloaded and prepared to /root/.cache/huggingface/datasets/cais___mmlu/college_computer_science/1.0.0/5c9df7080598ff5f0cc84b0e0519cfdc5516c84877d7893e547389485cfa78d8. Subsequent call

Above we tested on a subset of mmlu with zero-shot. If you are willing to wait a while, you can instead evaluate 5-shot on the entire MMLU dataset.

In [8]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
model_args = f"pretrained={model_name},load_in_4bit=True,bnb_4bit_compute_dtype=float16"
tasks = 'hendrycksTest-*'
!python lm-evaluation-harness/main.py \
  --model=hf-causal-experimental \
  --model_args=$model_args \
  --tasks=$tasks \
  --num_fewshot=5 \
  --batch_size=auto \
  --output_path="hendrycks/llama2.json" \
  --device=cuda:0

2023-07-23 18:09:39.218744: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Selected Tasks: ['hendrycksTest-abstract_algebra', 'hendrycksTest-anatomy', 'hendrycksTest-astronomy', 'hendrycksTest-business_ethics', 'hendrycksTest-clinical_knowledge', 'hendrycksTest-college_biology', 'hendrycksTest-college_chemistry', 'hendrycksTest-college_computer_science', 'hendrycksTest-college_mathematics', 'hendrycksTest-college_medicine', 'hendrycksTest-college_physics', 'hendrycksTest-computer_security', 'hendrycksTest-conceptual_physics', 'hendrycksTest-econometrics', 'hendrycksTest-electrical_engineering', 'hendrycksTest-elementary_mathematics', 'hendrycksTest-formal_logic', 'hendrycksTest-global_facts', 'hendrycksTest-high_school_biology', 'hendrycksTest-high_school_chemistry', 'hendrycksTest-high_school_computer_science', 'hendrycksTest-high_school_european_history', 'hendrycksTest-high_school_geography', 'hendrycksTest-high_school_government_and_